# Class 5: Statistical vs Visual EDA with Hawks Dataset

**Prerequisites:** Have actively worked through Tutorials 1 - 5.

---

## Learning Objectives

By the end of this class, students will be able to:
- Conduct systematic dataset exploration using pandas fundamentals
- Distinguish when to use statistical vs visual approaches in EDA workflows
- Investigate distributional questions methodically using both approaches
- Integrate pandas and Altair effectively for comprehensive analysis
- Apply decision frameworks for choosing analytical tools based on goals
- Discuss the role of bin sizes on histograms fidelity to the underlying data

---

## Environment Setup

In [1]:
import pandas as pd
import altair as alt
import numpy as np

alt.renderers.enable("default")

RendererRegistry.enable('default')

---

## Part 1: Meet the Hawks Dataset


In [2]:
# FILEPATH
# If on PL use this one
filepath = 'data/hawks.csv'

# If running locally on your machine use this one
filepath = 'https://raw.githubusercontent.com/kemiolamudzengi/dsci-320-datasets/main/hawks.csv'

# Load the hawks dataset
hawks = pd.read_csv(filepath)


### Basic Exploration Questions
Before we can get started, we need to get an overall sense of our dataset.

When presented with a new dataset, how do you start your exploration? 
What questions should you ask?
1. How big is the dataset (i.e. rows and columns)
2. How many attributes are there?
3. What are the data types of each attribute?
4. What is the missing data situation?
5. Which attributes seem to be a best starting spot for exploration?


Let's start by systematically exploring our new dataset using **pandas fundamentals**:

In [3]:

# Basic dataset metadata
print(f"Dataset shape: {hawks.shape}\n")


# Data types and detailed overview
hawks.info()

Dataset shape: (908, 20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 908 entries, 0 to 907
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    908 non-null    int64  
 1   Month         908 non-null    int64  
 2   Day           908 non-null    int64  
 3   Year          908 non-null    int64  
 4   CaptureTime   908 non-null    object 
 5   ReleaseTime   907 non-null    object 
 6   BandNumber    908 non-null    object 
 7   Species       908 non-null    object 
 8   Age           908 non-null    object 
 9   Sex           332 non-null    object 
 10  Wing          907 non-null    float64
 11  Weight        898 non-null    float64
 12  Culmen        901 non-null    float64
 13  Hallux        902 non-null    float64
 14  Tail          908 non-null    int64  
 15  StandardTail  571 non-null    float64
 16  Tarsus        75 non-null     float64
 17  WingPitFat    77 non-null     float64
 18  Keel

In [4]:
# First few rows to understand the data structure
hawks.head()
# or you can use sample so it doesn't just give what is at the top
hawks.sample(10)

,Unnamed: 0,Month,Day,Year,CaptureTime,ReleaseTime,BandNumber,Species,Age,Sex,Wing,Weight,Culmen,Hallux,Tail,StandardTail,Tarsus,WingPitFat,KeelFat,Crop
424,425,9,21,1999,12:12,,1213-58901,SS,I,F,197.0,165.0,11.9,9.6,150,152.0,NaN,NaN,1.0,0.00
228,229,9,27,1995,12:50,,1387-79178,RT,I,NaN,411.0,1240.0,26.8,32.7,276,NaN,NaN,NaN,NaN,NaN
404,405,11,9,1998,1:31,,1387-92183,RT,A,NaN,391.0,1097.0,27.9,32.8,217,NaN,NaN,NaN,NaN,NaN
237,238,10,4,1995,12:54,,1387-79183,RT,A,NaN,345.0,1000.0,26.4,30.1,200,NaN,NaN,NaN,NaN,NaN
243,244,10,7,1995,12:00,,1387-79185,RT,I,NaN,313.0,930.0,25.7,28.9,215,NaN,NaN,NaN,NaN,NaN
248,249,10,10,1995,14:25,,1705-24608,CH,I,F,259.0,470.0,16.6,23.4,221,NaN,NaN,NaN,NaN,NaN
502,503,9,9,2000,9:38,,1142-18229,SS,I,M,161.0,95.0,10.1,11.3,135,129.0,NaN,NaN,4.0,0.00
189,190,11,2,1994,14:52,,1387-79159,RT,A,NaN,377.0,1055.0,27.0,29.1,200,NaN,NaN,NaN,NaN,NaN
731,732,9,30,2002,12:46,,1207-72669,RT,I,NaN,403.0,1160.0,29.0,32.3,235,236.0,NaN,NaN,3.0,0.25
238,239,10,5,1995,11:08,,1705-24607,CH,I,F,273.0,530.0,19.2,24.7,213,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Missing data landscape
missing_summary = hawks.isnull().sum()
print("Missing values per column:")
print(missing_summary[missing_summary > 0])

print(f"\nTotal missing values: {missing_summary.sum()}") # .sum() allows us to see relative valid data

print(f"Percentage of complete rows: {(hawks.dropna().shape[0] / hawks.shape[0] * 100):.1f}%") # tells us about percentage of complete rows! Really useful.

Missing values per column:
ReleaseTime       1
Sex             576
Wing              1
Weight           10
Culmen            7
Hallux            6
StandardTail    337
Tarsus          833
WingPitFat      831
KeelFat         341
Crop            343
dtype: int64

Total missing values: 3286
Percentage of complete rows: 3.9%


#### Missing Data Issue

Only 3.9% of our rows have all the attributes. 

**YIKES**, this is not a great start. 
Next week we will delve deeper into doing a missing data analysis. For right now we will take the avoidance approach and just exclude the most offending columns (i.e. Sex, Tarsus, WingPitFat, KeelFat, Crop, StandardTail). I'm also going to remove some other columns just so we can have a targeted exploration.  

In [6]:
cols_to_drop = [
    'Unnamed: 0', 'ReleaseTime', 'StandardTail', 'Tarsus', 'Sex',
    'KeelFat', 'Crop', 'BandNumber', 'CaptureTime', 'WingPitFat', 'Day'
]

hawks_clean = (
    hawks
    .drop(columns=cols_to_drop, errors='ignore')
    .rename(columns=lambda x: x.strip().lower()) # we are used to lower-case attribute names, so drop the title case
).copy()

print(f"Cleaned dataset shape: {hawks_clean.shape}")


Cleaned dataset shape: (908, 9)


In [7]:
# Let's do a missing data analysis again

# Missing data landscape
missing_summary = hawks_clean.isnull().sum()
print("Missing values per column:")
print(missing_summary[missing_summary > 0])

print(f"\nTotal missing values: {missing_summary.sum()}")
print(f"Percentage of complete rows: {(hawks_clean.dropna().shape[0] / hawks_clean.shape[0] * 100):.1f}%")


Missing values per column:
wing       1
weight    10
culmen     7
hallux     6
dtype: int64

Total missing values: 24
Percentage of complete rows: 98.1%


Okay so we went from 3% to 98%, WE ARE AWESOME!!!.
Now it is worth saying that in industry you can't just ignore all the attributes that have missing values, so we will revisit this later. 


Here are the attributes that we decided to focus on today

### Hawks Dataset Data Card

| Column       | Description   | Attribute Type |                                                               
|--------------|---------------------------------------------------------|------|
| month | Month of capture | Temporal |
| year | Year of capture | Temporal |
| species      | hawk's species     | Nominal |
| age          | age group: A=Adult or I=Immature | Ordinal  |
| wing         | Length (in mm) of primary wing feather from tip to wrist it attaches to | Quantitative |
| weight       | Body weight (in gm)    | Quantitative |
| culmen       | Length (in mm) of the upper bill from the tip to where it bumps into the fleshy part of the bird | Quantitative |
| hallux       | Length (in mm) of the killing talon      | Quantitative |
| tail         | Measurement (in mm) related to the length of the tail    | Quantitative         |

### Dataset Context & Discussion Points

**&#128269; What type of data collection is this?**
- Research study of captured and measured hawks
- Multiple numeric measurements (weight, wing span, etc.)
- Categorical classifications (species, sex, age)

**&#129300;  What variables might be related?**
- Size measurements probably correlate
- Species likely affects size significantly  
- Sex differences within species?

**⚠️ Data quality considerations:**
- Missing data patterns (detailed analysis in Class 6)
- Potential outliers in measurements
- Measurement precision and errors

---


## Part 2: The Distributional Question 

### Setting Up the Core EDA Question

> **In typical EDA workflows, we ask: "How does a quantitative variable distribute across our observed data?"**

This is one of the most fundamental questions in exploratory data analysis. Let's investigate the distribution of hawk weights.

### The Standard Approach: Histogram

<div style="border-left: 5px solid #007BFF; padding: 15px; background-color: #F0F8FF; border-radius: 8px;">

<h2>Hawk Weight Histogram</h2>

<p><strong>Exploratory Question:</strong>  
<em>What is the distribution of hawk weights in the dataset?</em></p>

<p><strong>VIZ TASK:</strong> Create a histogram showing the distribution of hawk weights.</p>

<h3>Chart Specification:</h3>
<ul>
<li>Use <code>mark_bar</code> with <code>binSpacing=0</code> to ensure no gaps between bins.</li>
<li>Encode <code>weight</code> on the <strong>x channel</strong> as <code>quantitative</code> with a binning specification (<code>maxbins=25</code>).</li>
<li>Encode <code>count()</code> on the <strong>y channel</strong> as <code>quantitative</code>, titled <em>Number of Hawks</em>.</li>
<li>Set the chart <strong>width</strong> to 400 pixels and <strong>height</strong> to 300 pixels.</li>
<li>Give the chart the title <em>Hawk Weight Distribution</em>.</li>
</ul>

</div>


In [10]:
# Basic distribution exploration using histogram
hawk_histogram = alt.Chart(hawks_clean).mark_bar(binSpacing = 0).encode(x = alt.X('weight:Q', bin = alt.Bin(maxbins = 25), title = "Weight (g)"),
y = alt.Y('count():Q', title = "Number of Hawks")).properties(width = 400, height = 300, title = 'Hawk Weight Distributiogn')


# Show the chart
hawk_histogram

alt.Chart(...)

### Observation
What do you notice about the distribution?
...
How would you characterize this distribution?
...

## Part 3: The Statistical Approach 

### Fundamental Pandas EDA Techniques

Let's systematically investigate using **computational summaries**:

In [ ]:
# 1. Overall distribution characteristics
print("Overall weight distribution:")
print(hawks_clean['weight'].describe()) # we want to specify which *quantitative variable* we want to hone into!

Overall weight distribution:
count     898.000000
mean      772.080178
std       462.311760
min        56.000000
25%       185.000000
50%       970.000000
75%      1120.000000
max      2030.000000
Name: weight, dtype: float64


In [12]:
# 2. Investigate potential grouping variables
print("Available categorical variables:")
categorical_cols = hawks_clean.select_dtypes(include=['object']).columns

for col in categorical_cols:
    print(f"\n{col}: {hawks_clean[col].nunique()} unique values")
    print(hawks_clean[col].value_counts())

Available categorical variables:

species: 3 unique values
species
RT    577
SS    261
CH     70
Name: count, dtype: int64

age: 2 unique values
age
I    684
A    224
Name: count, dtype: int64


In [13]:
# 3. Group-wise statistical analysis
# Hypothesis 1: Does species explain the distribution?
species_stats = hawks_clean.groupby('species')['weight'].describe()
print("Weight statistics by species:")
print(species_stats)

Weight statistics by species:
         count         mean         std    min    25%     50%      75%     max
species                                                                       
CH        70.0   420.485714  162.031643   56.0  335.0   377.5   505.00  1119.0
RT       572.0  1094.430070  189.210250  101.0  980.0  1070.0  1210.00  2030.0
SS       256.0   147.968750   80.652675   85.0  100.0   155.0   177.75  1094.0


In [ ]:
# 3. Group-wise statistical analysis
# Hypothesis 2: Does age explain the distribution?
age_stats = hawks_clean.groupby('age')['weight'].describe()
print("Weight statistics by age:")
print(age_stats)

### 🔍 Observation: Statistical Detective Findings

**From the statistical analysis, what do you notice?**

> 1. The `RT` species has the greatest mean weight. Very high max as well.
> 2. Though there's only 70 `CH` species, their mean weight is 420g compared to the 256 `SS` with a mean weight of 147 g. 
> 3. `SS` species seems to have a lot less variation in terms of their weight compared to the two other species.
> 4. There doesn seem to have a lot of difference in Adult / Immature hawk count, the mean weight between the two isn't much different, suggesting age probably didn't play a huge role in making the bi-modal distribution we observed.
> 
> 😱

---



## Part 4: The Visual  Approach 

### Visual Investigation Using Faceting

Now let's investigate the **same question** using visual exploration

<br>

<div style="border-left: 5px solid #007BFF; padding: 15px; background-color: #F0F8FF; border-radius: 8px;">

<h2>Density of Hawk Weights by Species</h2>

<p><strong>Exploratory Question:</strong>  
<em>Does species explain the multimodal distribution of hawk weights?</em></p>

<p><strong>VIZ TASK:</strong> Create a density plot of hawk weights, grouped by species, to compare distributions.</p>

<h3>Chart Specification:</h3>
<ul>
<li>Transform the data using <code>transform_density</code> on <code>weight</code>, grouping by <code>species</code>.</li>
<li>Use <code>mark_area</code> for smooth density curves.</li>
<li>Encode <code>weight</code> on the <strong>x channel</strong> as <code>quantitative</code>, with the title <em>Weight (g)</em>.</li>
<li>Encode <code>density</code> on the <strong>y channel</strong> as <code>quantitative</code>, with the title <em>Density</em>.</li>
<li>Encode <code>species</code> on the <strong>color channel</strong> as <code>nominal</code>, with the legend title <em>Species</em>.</li>
<li>Set the chart <strong>width</strong> to 400 pixels and <strong>height</strong> to 200 pixels.</li>
<li>Give the chart the title <em>Density of Hawk Weights by Species</em>.</li>
</ul>

<h3>Styling Specifications:</h3>
<ul>
<li>Set <code>opacity</code> of the density areas to <code>0.7</code> for overlap visibility.</li>
</ul>

</div>


In [18]:
# Visual hypothesis testing: Does species explain the multimodal pattern?
density_by_species = alt.Chart(hawks_clean).transform_density('weight', groupby = ['species'], as_=['weight', 'density']).mark_area(opacity = 0.7).encode(
                     x = alt.X('weight:Q', title = 'Weight (g)'),
                     y = alt.Y('density:Q', title = 'Density'),
                     color = alt.Color('species:N', title = 'Species'),
                     ).properties(width = 500, height = 300, title = 'Density of Hawk Weights by Species')


# Show the chart

density_by_species

alt.Chart(...)

### Observation
What do you notice from the layered density plot?

...


<div style="border-left: 5px solid #007BFF; padding: 15px; background-color: #F0F8FF; border-radius: 8px;">

<h2>Density of Hawk Weights by Age</h2>

<p><strong>Exploratory Question:</strong>  
<em>Does age explain the multimodal distribution of hawk weights?</em></p>

<p><strong>VIZ TASK:</strong> Create a density plot of hawk weights, grouped by age, to compare distributions across age categories.</p>

<h3>Chart Specification:</h3>
<ul>
<li>Transform the data using <code>transform_density</code> on <code>weight</code>, grouping by <code>age</code>.</li>
<li>Use <code>mark_area</code> to display smooth density curves.</li>
<li>Encode <code>weight</code> on the <strong>x channel</strong> as <code>quantitative</code>, with the axis title <em>Weight (g)</em>.</li>
<li>Encode <code>density</code> on the <strong>y channel</strong> as <code>quantitative</code>, with the axis title <em>Density</em>.</li>
<li>Encode <code>age</code> on the <strong>color channel</strong> as <code>ordinal</code>, with the legend title <em>Age</em>.</li>
<li>Set the chart <strong>width</strong> to 400 pixels and <strong>height</strong> to 200 pixels.</li>
<li>Give the chart the title <em>Density of Hawk Weights by Age</em>.</li>
</ul>

<h3>Styling Specifications:</h3>
<ul>
<li>Set <code>opacity</code> of the density areas to <code>0.7</code> for overlap visibility.</li>
</ul>

</div>


In [19]:
# Visual hypothesis testing: Does age explain the multimodal pattern?
density_by_age = alt.Chart(hawks_clean).transform_density('weight', groupby = ['age'], as_=['weight', 'density']).mark_area(opacity = 0.7).encode(
                     x = alt.X('weight:Q', title = 'Weight (g)'),
                     y = alt.Y('density:Q', title = 'Density'),
                     color = alt.Color('age:O', title = 'Age'),
                     ).properties(width = 500, height = 300, title = 'Density of Hawk Weights by Age')



density_by_age

alt.Chart(...)

### Observation
What do you notice from the layered density plot?

...


### 🎨 Observation: Visual Discovery Process

**What do you immediately notice from this visualization?**

...

---


## Part 5: Integration - The Best of Both Worlds 

### Combining Statistical Precision with Visual Insight

The real power comes from using **both approaches together** for deeper understanding:

In [ ]:
# Start with visual exploration → Species clearly explains multimodal distribution
# Now use statistics to quantify the visual findings

species_summary = hawks_clean.groupby('species')['weight'].agg([
    'count', 'mean', 'std', 'min', 'max'
]).round(1)

print("Quantified species differences:")
print(species_summary)

In [ ]:
# The visual analysis suggested we should investigate age differences too
# Let's get statistical confirmation:

detailed_stats = hawks_clean.groupby(['species', 'age'])['weight'].agg([
    'count', 'mean', 'std'
]).round(1)

print("Detailed breakdown by species AND age:")
print(detailed_stats)



## Part 6: Histogram Deep Dive - The Power of Binning
### *When Visualization Choices Change Your Story*

We've created histograms earlier, but have you considered how your binning choices affect the insights you discover?

### **The Binning Experiment**

In [ ]:
# Same data, different bin sizes
bin_sizes = [5, 10, 20, 30, 50]

charts = []
for bins in bin_sizes:
    chart = alt.Chart(hawks_clean).mark_bar().encode(
        x=alt.X('weight:Q', 
               bin=alt.BinParams(maxbins=bins), 
               title=f'Weight (g) - {bins} bins'),
        y=alt.Y('count()', title='Count'),
        color=alt.value('steelblue')
    ).properties(
        width=200,
        height=150,
        title=f"{bins} Bins"
    )
    charts.append(chart)

# Arrange in a grid
binning_comparison = alt.vconcat(
    alt.hconcat(charts[0], charts[1], charts[2]),
    alt.hconcat(charts[3], charts[4])
).properties(
    title="How Binning Affects Pattern Detection"
)

binning_comparison

### Observation
What do you notice about the different binning sizes?


### **Binning by Groups**

<div style="border-left: 5px solid #007BFF; padding: 15px; background-color: #F0F8FF; border-radius: 8px;">

<h2>Weight Distribution by Species</h2>

<p><strong>Exploratory Question:</strong>  
<em>Do different hawk species require different binning strategies when examining weight distributions?</em></p>

<p><strong>VIZ TASK:</strong> Create small multiple histograms to compare the weight distributions across species.</p>

<h3>Chart Specification:</h3>
<ul>
<li>Use <code>mark_bar</code> to create histograms of hawk weights.</li>
<li>Encode <code>weight</code> on the <strong>x channel</strong> as <code>quantitative</code>, applying binning with <code>maxbins=15</code>. Title the axis <em>Weight (g)</em>.</li>
<li>Encode <code>count()</code> on the <strong>y channel</strong> as <code>quantitative</code>, with the axis title <em>Count</em>.</li>
<li>Encode <code>species</code> on the <strong>color channel</strong> as <code>nominal</code>, with the legend title <em>Species</em>.</li>
<li>Use the <strong>column channel</strong> for <code>species</code> so that each species appears as a separate small multiple histogram, with the title <em>Species</em>.</li>
<li>Set the overall chart title to <em>Weight Distribution by Species</em>.</li>
</ul>

<h3>Styling Specifications:</h3>
<ul>
<li>Set the <strong>width</strong> of each facet to 200 pixels and <strong>height</strong> to 150 pixels.</li>
<li>Apply a distinct color for each species category (Altair’s default categorical palette).</li>
</ul>

</div>


In [ ]:
# Do different species need different binning strategies?
species_histograms = ...

# Show the plot
species_histograms

### Observation
Change the number of bins (i.e, maxbins value), when you look at each of the species, which ones benefit from a specific bin size. 





## Reflection 

### Key Takeaways Discussion

**🤔 Reflection Questions:**

1. **Complementary approaches:** How did statistics and visualization reveal different aspects of the same data?

2. **Discovery process:** Which approach led you to the "aha moment" about species differences?

3. **Efficiency trade-offs:** When might a 30-second `.describe()` be better than a 5-minute faceted chart?

4. **Integration benefits:** How did combining both approaches give deeper insights?


### Quick Practice Exercise

**Before we wrap up:**

> *"If you wanted to investigate whether wing span varies by species, would you start with pandas or Altair? Why?"*

**Think about:**
- Your analytical goal
- Time constraints  
- Expected complexity
- Audience for results

---

## Summary

Today we explored the **fundamental tension** in exploratory data analysis:

**🔢 Statistical EDA:** Fast, precise, hypothesis-driven  
**🎨 Visual EDA:** Intuitive, exploratory, communication-focused

**🎯 The key insight:** They're not competing approaches - they're complementary tools that together provide comprehensive understanding of your data.
